In [1]:
#=============================导入库函数===============================#
import time
from pynq import Overlay
import numpy as np
from pynq import Xlnk
import struct
from scipy.misc import imread
import cv2

In [2]:
#=============================部署Xlnk核Overlay===============================#
xlnk=Xlnk() #调用pynq板子特有的库，用于分配地址
ol=Overlay("./bit/whale.bit") #加载生成的比特流文件
ol.ip_dict #打印ip中的属性
ol.download() #下载到板子
conv=ol.Conv_0 #ip的一个属性
pool=ol.Pool_0 #ip的一个属性
print("Overlay download finish")

Overlay download finish


In [3]:
#=============================定义卷积函数和池化函数=============================
def RunConv(conv,Kx,Ky,Sx,Sy,mode,relu_en,feature_in,W,bias,feature_out):
    conv.write(0x10,feature_in.shape[2])
    conv.write(0x18,feature_in.shape[0])
    conv.write(0x20,feature_in.shape[1])
    conv.write(0x28,feature_out.shape[2])
    conv.write(0x30,Kx)
    conv.write(0x38,Ky)
    conv.write(0x40,Sx)
    conv.write(0x48,Sy)
    conv.write(0x50,mode)
    conv.write(0x58,relu_en)
    conv.write(0x60,feature_in.physical_address)
    conv.write(0x68,W.physical_address)
    conv.write(0x70,bias.physical_address)
    conv.write(0x78,feature_out.physical_address)
    conv.write(0, (conv.read(0)&0x80)|0x01 )
    tp=conv.read(0)
    while not ((tp>>1)&0x1): #判断是否计算完成
        tp=conv.read(0)
    #print(tp)

def RunPool(pool,Kx,Ky,mode,feature_in,feature_out):
    pool.write(0x10,feature_in.shape[2])
    pool.write(0x18,feature_in.shape[0])
    pool.write(0x20,feature_in.shape[1])
    pool.write(0x28,Kx)
    pool.write(0x30,Ky)
    pool.write(0x38,mode)
    pool.write(0x40,feature_in.physical_address)
    pool.write(0x48,feature_out.physical_address)
    pool.write(0, (pool.read(0)&0x80)|0x01 )
    while not ((pool.read(0)>>1)&0x1): #判断是否计算完成
        pass 

In [4]:
# -*- coding: UTF-8 -*-
# https://blog.csdn.net/u013832707/article/details/73608504
import sys

class ShowProcess():
    """
    显示处理进度的类
    调用该类相关函数即可实现处理进度的显示
    """
    i = 0 # 当前的处理进度
    max_steps = 0 # 总共需要处理的次数
    max_arrow = 50 #进度条的长度
    infoDone = 'done'

    # 初始化函数，需要知道总共的处理次数
    def __init__(self, max_steps, infoDone = 'Done'):
        self.max_steps = max_steps
        self.i = 0
        self.infoDone = infoDone

    # 显示函数，根据当前的处理进度i显示进度
    # 效果为[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
    def show_process(self, i=None):
        if i is not None:
            self.i = i
        else:
            self.i += 1
        num_arrow = int(self.i * self.max_arrow / self.max_steps) #计算显示多少个'>'
        num_line = self.max_arrow - num_arrow #计算显示多少个'-'
        percent = self.i * 100.0 / self.max_steps #计算完成进度，格式为xx.xx%
        process_bar = '[' + '>' * num_arrow + '-' * num_line + ']'\
                      + '%.2f' % percent + '%' + '\r' #带输出的字符串，'\r'表示不换行回到最左边
        sys.stdout.write(process_bar) #这两句打印字符到终端
        sys.stdout.flush()
        if self.i >= self.max_steps:
            self.close()

    def close(self):
        print('')
        print(self.infoDone)
        self.i = 0


# 1 ship

In [12]:
#=============================给定各个参数的值===============================#
# ship
IN_WIDTH1=32 #图片的宽度
IN_HEIGHT1=32 #图片的高度
IN_CH1=1 # 输入层通道
OUT_CH1=16  # 第一层卷积输出通道
OUT_CH2=32  # 第二层卷积输出通道
KERNEL_WIDTH3=8 # 第二层池化输出图像宽度
KERNEL_HEIGHT3=8 # 第二层池化输出图像高度
OUT_CH3=128 # 第一层全连接输出通道
OUT_CH4=5 #最终输出通道

#Conv1 卷积1

print('IN_CH1 is',IN_CH1)
KERNEL_WIDTH1=3 #卷积核宽度
KERNEL_HEIGHT1=3 #卷积核高度
X_STRIDE1=1 #移动步长x
Y_STRIDE1=1 #移动步长y
RELU_EN1 = 1 
MODE1=1    #0:VALID, 1:SAME #选择padding
#MODE1=0    #0:VALID, 1:SAME #选择padding
if(MODE1):
    X_PADDING1=int((KERNEL_WIDTH1-1)/2)
    Y_PADDING1=int((KERNEL_HEIGHT1-1)/2)
else:
    X_PADDING1=0
    Y_PADDING1=0
    
OUT_WIDTH1=int((IN_WIDTH1+2*X_PADDING1-KERNEL_WIDTH1)/X_STRIDE1+1) #输出的宽度
OUT_HEIGHT1=int((IN_HEIGHT1+2*Y_PADDING1-KERNEL_HEIGHT1)/Y_STRIDE1+1) #输出的高度

#Pool1 池化1
MODE11=2  #mode: 0:MEAN, 1:MIN, 2:MAX #选择最大池化
IN_WIDTH11=OUT_WIDTH1 #输入宽度为第一层卷积输出的宽度
IN_HEIGHT11=OUT_HEIGHT1 #输入高度为第一层卷积输出的高度
IN_CH11=OUT_CH1 #

KERNEL_WIDTH11=2
KERNEL_HEIGHT11=2

OUT_CH11=IN_CH11
OUT_WIDTH11=int(IN_WIDTH11/KERNEL_WIDTH11)
OUT_HEIGHT11=int(IN_HEIGHT11/KERNEL_HEIGHT11)

#Conv2 卷积2
IN_WIDTH2=OUT_WIDTH11
IN_HEIGHT2=OUT_HEIGHT11
IN_CH2=OUT_CH11
print('IN_CH2 is',IN_CH2)

KERNEL_WIDTH2=3
KERNEL_HEIGHT2=3
X_STRIDE2=1
Y_STRIDE2=1

RELU_EN2=1
MODE2=1  #0:VALID, 1:SAME #选择padding
#MODE2=0  #0:VALID, 1:SAME #选择padding
if(MODE2):
    X_PADDING2=int((KERNEL_WIDTH2-1)/2)
    Y_PADDING2=int((KERNEL_HEIGHT2-1)/2)
else:
    X_PADDING2=0
    Y_PADDING2=0

OUT_WIDTH2=int((IN_WIDTH2+2*X_PADDING2-KERNEL_WIDTH2)/X_STRIDE2+1)
OUT_HEIGHT2=int((IN_HEIGHT2+2*Y_PADDING2-KERNEL_HEIGHT2)/Y_STRIDE2+1)

#Pool2 池化2
MODE21=2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_WIDTH21=OUT_WIDTH2
IN_HEIGHT21=OUT_HEIGHT2
IN_CH21=OUT_CH2

KERNEL_WIDTH21=2
KERNEL_HEIGHT21=2

OUT_CH21=IN_CH21
OUT_WIDTH21=int(IN_WIDTH21/KERNEL_WIDTH21)
OUT_HEIGHT21=int(IN_HEIGHT21/KERNEL_HEIGHT21)

#Fc1 全连接1 由于全连接和卷积层的计算原理相同，所以可利用卷积层的ip实现全连接
IN_WIDTH3=OUT_WIDTH21
IN_HEIGHT3=OUT_HEIGHT21
IN_CH3=OUT_CH21
print('IN_CH3 is',IN_CH3)

X_STRIDE3=1
Y_STRIDE3=1
RELU_EN3=1
MODE3=0  #0:VALID, 1:SAME #选择不padding
if(MODE3):
    X_PADDING3=int((KERNEL_WIDTH3-1/2))
    Y_PADDING3=int((KERNEL_HEIGHT3-1)/2)
else:
    X_PADDING3=0
    Y_PADDING3=0

OUT_WIDTH3=int((IN_WIDTH3+2*X_PADDING3-KERNEL_WIDTH3)/X_STRIDE3+1)
OUT_HEIGHT3=int((IN_HEIGHT3+2*Y_PADDING3-KERNEL_HEIGHT3)/Y_STRIDE3+1)

#Fc2 全连接2
IN_WIDTH4=OUT_WIDTH3
IN_HEIGHT4=OUT_HEIGHT3
IN_CH4=OUT_CH3
print('IN_CH4 is',IN_CH4)

KERNEL_WIDTH4=1
KERNEL_HEIGHT4=1
X_STRIDE4=1
Y_STRIDE4=1

RELU_EN4=1
MODE4=0  #0:VALID, 1:SAME
if(MODE4):
    X_PADDING4=int((KERNEL_WIDTH4-1/2))
    Y_PADDING4=int((KERNEL_HEIGHT4-1)/2)
else:
    X_PADDING4=0
    Y_PADDING4=0

OUT_WIDTH4=int((IN_WIDTH4+2*X_PADDING4-KERNEL_WIDTH4)/X_STRIDE4+1)
OUT_HEIGHT4=int((IN_HEIGHT4+2*Y_PADDING4-KERNEL_HEIGHT4)/Y_STRIDE4+1)


IN_CH1 is 1
IN_CH2 is 16
IN_CH3 is 32
IN_CH4 is 128


In [13]:
#=============================分配存储空间给各个参数===============================#
#input image
image=xlnk.cma_array(shape=(IN_HEIGHT1,IN_WIDTH1,IN_CH1),cacheable=0,dtype=np.float32) #分配空间给image存储图片
#conv1
W_conv1=xlnk.cma_array(shape=(KERNEL_HEIGHT1,KERNEL_WIDTH1,IN_CH1,OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积权重的存储空间
b_conv1=xlnk.cma_array(shape=(OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积偏置的存储空间
h_conv1=xlnk.cma_array(shape=(OUT_HEIGHT1,OUT_WIDTH1,OUT_CH1),cacheable=0,dtype=np.float32) #给池化分配空间
h_pool1=xlnk.cma_array(shape=(OUT_HEIGHT11,OUT_WIDTH11,OUT_CH11),cacheable=0,dtype=np.float32) #给池化分配空间
#conv2
W_conv2=xlnk.cma_array(shape=(KERNEL_HEIGHT2,KERNEL_WIDTH2,IN_CH2,OUT_CH2),cacheable=0,dtype=np.float32)
b_conv2=xlnk.cma_array(shape=(OUT_CH2),cacheable=0,dtype=np.float32)
h_conv2=xlnk.cma_array(shape=(OUT_HEIGHT2,OUT_WIDTH2,OUT_CH2),cacheable=0,dtype=np.float32)
h_pool2=xlnk.cma_array(shape=(OUT_HEIGHT21,OUT_WIDTH21,OUT_CH21),cacheable=0,dtype=np.float32)
#fc1
W_fc1=xlnk.cma_array(shape=(KERNEL_HEIGHT3, KERNEL_WIDTH3, IN_CH3, OUT_CH3),cacheable=0,dtype=np.float32)
b_fc1=xlnk.cma_array(shape=(OUT_CH3),cacheable=0,dtype=np.float32)
h_fc1=xlnk.cma_array(shape=(OUT_HEIGHT3,OUT_WIDTH3,OUT_CH3),cacheable=0,dtype=np.float32)
#fc2
W_fc2=xlnk.cma_array(shape=(KERNEL_HEIGHT4, KERNEL_WIDTH4, IN_CH4, OUT_CH4),cacheable=0,dtype=np.float32)
b_fc2=xlnk.cma_array(shape=(OUT_CH4),cacheable=0,dtype=np.float32)
h_fc2=xlnk.cma_array(shape=(OUT_HEIGHT4,OUT_WIDTH4,OUT_CH4),cacheable=0,dtype=np.float32)


In [14]:
#=============================初始化：读取训练好的参数，并写入分配好的板上存储空间===============================#
w_conv1=np.load('./record/ship/tensor1.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv1=np.load('./record/ship/tensor2.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv1=np.transpose(w_conv1,(1,2,3,0))
for i in range(KERNEL_HEIGHT1): #将数据写入板子
    for j in range(KERNEL_WIDTH1):
        for k in range(IN_CH1):
        	for l in range(OUT_CH1):
        		W_conv1[i][j][k][l]=w_conv1[i][j][k][l]
for i in range(OUT_CH1): #将数据写入板子
#     b_conv1[i]=B_conv1[i]
    b_conv1[i]=0
#=================================================    
w_conv2=np.load('./record/ship/tensor3.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv2=np.load('./record/ship/tensor4.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv2=np.transpose(w_conv2,(1,2,3,0))
for i in range(KERNEL_HEIGHT2):
    for j in range(KERNEL_WIDTH2):
        for k in range(IN_CH2):
        	for l in range(OUT_CH2):
        		W_conv2[i][j][k][l]=w_conv2[i][j][k][l]
for i in range(OUT_CH2):
#     b_conv2[i]=B_conv2[i]
    b_conv2[i]=0
#=================================================    
w_fc1=np.load('./record/ship/tensor5.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc1=np.load('./record/ship/tensor6.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc1=np.transpose(w_fc1,(1,0))
w_fc1=w_fc1.reshape((KERNEL_HEIGHT3,KERNEL_WIDTH3,IN_CH3,OUT_CH3))
for i in range(KERNEL_HEIGHT3):
    for j in range(KERNEL_WIDTH3):
        for k in range(IN_CH3):
        	for l in range(OUT_CH3):
        		W_fc1[i][j][k][l]=w_fc1[i][j][k][l]
for i in range(OUT_CH3):
#     b_fc1[i]=B_fc1[i]
    b_fc1[i]=0
#=================================================    
w_fc2=np.load('./record/ship/tensor7.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc2=np.load('./record/ship/tensor8.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc2=np.transpose(w_fc2,(1,0))
w_fc2=w_fc2.reshape((KERNEL_HEIGHT4,KERNEL_WIDTH4,IN_CH4,OUT_CH4))
for i in range(KERNEL_HEIGHT4):
    for j in range(KERNEL_WIDTH4):
        for k in range(IN_CH4):
        	for l in range(OUT_CH4):
        		W_fc2[i][j][k][l]=w_fc2[i][j][k][l]                                      
for i in range(OUT_CH4):
#     b_fc2[i]=B_fc2[i]
    b_fc2[i]=0
#=================================================    
print("Finish initial")

Finish initial


In [21]:
# 单张图片测试
img = cv2.imread("./test_data/ship.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
#start_time,end_time,time_use=0,0,0
for i in range(IN_HEIGHT1):
    for j in range(IN_WIDTH1):
        image[i][j][0]=img[i][j]

#调用卷积ip、池化ip，实现硬件加速
#由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
start_time=time.time()
#conv1
RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
conv1_time=time.time()
RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
pool1_time=time.time()
# conv2
RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
conv2_time=time.time()
RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
pool2_time=time.time()
# fc1 
RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
fc1_time=time.time()
# fc2
RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
fc2_time=time.time()

result=h_fc2[0][0].argmax() 
result_name=name[result]

# 显示图像    
print('result:',result_name)
print('conv1_time:',np.round(conv1_time-start_time,4),'s')
print('pool1_time:',np.round(pool1_time-conv1_time,4),'s')
print('conv2_time:',np.round(conv2_time-pool1_time,4),'s')
print('pool2_time:',np.round(pool2_time-conv2_time,4),'s')
print('fc1  _time:',np.round(fc1_time-pool2_time,4),'s')
print('fc2  _time:',np.round(fc2_time-fc1_time,4),'s')
print('all  _time:',np.round(fc2_time-start_time,4),'s')

result: 游轮
conv1_time: 0.0312 s
pool1_time: 0.0019 s
conv2_time: 0.2197 s
pool2_time: 0.0016 s
fc1  _time: 0.0523 s
fc2  _time: 0.0015 s
all  _time: 0.3082 s


In [4]:
#=============================手写数字识别===============================#
def cnn_mnist_test(img):
    #start_time,end_time,time_use=0,0,0
    for i in range(IN_HEIGHT1):
        for j in range(IN_WIDTH1):
            image[i][j][0]=img[i][j]

    #调用卷积ip、池化ip，实现硬件加速
    #由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
    start_time=time.time()
	#conv1
    RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
    RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
	# conv2
    RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
    RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
	# fc1 
    RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
	# fc2
    RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
    end_time=time.time()

    result=h_fc2[0][0].argmax() 
    time_use=end_time-start_time
    return(result)

In [19]:
ship_test_image_label=list()
ship_train_image_label=list()
f1=open('./dataset/ship_test_image_label_6_14.txt','r')
f2=open('./dataset/ship_train_image_label_6_14.txt','r')
for i in f1.readlines():
    ship_test_image_label.append(i.split('\n')[0].split(' '))
for i in f2.readlines():
    ship_train_image_label.append(i.split('\n')[0].split(' '))
f1.close()
f2.close()

In [60]:
ADD_Accurate=0
process_bar = ShowProcess(len(ship_test_image_label), 'Processing Finished')
for i in range(len(ship_test_image_label)):
    img = cv2.imread("./dataset/ship32x32/"+ship_test_image_label[i][0], cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
    res=cnn_mnist_test(img)
    if(res==int(ship_test_image_label[i][1])):
        ADD_Accurate+=1
    process_bar.show_process()


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [61]:
print('正确率:',ADD_Accurate,'/',len(ship_test_image_label),'=',np.round(ADD_Accurate/len(ship_test_image_label)*100,2))

正确率: 485 : 1249 = 38.83


In [51]:
ADD_Accurate=0
process_bar = ShowProcess(len(ship_train_image_label), 'Processing Finished')
for i in range(len(ship_train_image_label)):
    img = cv2.imread("./dataset/ship32x32/"+ship_train_image_label[i][0], cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
    res=cnn_mnist_test(img)
    if(res==int(ship_train_image_label[i][1])):
        ADD_Accurate+=1
    process_bar.show_process()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
正确数: 2516
正确率: 50.29


In [55]:
print('正确率:',ADD_Accurate,':',len(ship_train_image_label),'=',np.round(ADD_Accurate/len(ship_train_image_label)*100,2))

正确率: 2516 : 5003 = 50.29


In [ ]:
name=['0','1','2','3','4']# 游轮

In [29]:
# 单张图片测试
img = cv2.imread("./dataset/ship32x32/2876167.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
#start_time,end_time,time_use=0,0,0
for i in range(IN_HEIGHT1):
    for j in range(IN_WIDTH1):
        image[i][j][0]=img[i][j]

#调用卷积ip、池化ip，实现硬件加速
#由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
start_time=time.time()
#conv1
RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
conv1_time=time.time()
RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
pool1_time=time.time()
# conv2
RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
conv2_time=time.time()
RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
pool2_time=time.time()
# fc1 
RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
fc1_time=time.time()
# fc2
RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
fc2_time=time.time()

result=h_fc2[0][0].argmax() 
result_name=name[result]

# 显示图像    
print('result:',result_name)
print('conv1_time:',np.round(conv1_time-start_time,4),'s')
print('pool1_time:',np.round(pool1_time-conv1_time,4),'s')
print('conv2_time:',np.round(conv2_time-pool1_time,4),'s')
print('pool2_time:',np.round(pool2_time-conv2_time,4),'s')
print('fc1  _time:',np.round(fc1_time-pool2_time,4),'s')
print('fc2  _time:',np.round(fc2_time-fc1_time,4),'s')
print('all  _time:',np.round(fc2_time-start_time,4),'s')

# 2 whale

In [22]:
# whale
IN_WIDTH1=64 #图片的宽度
IN_HEIGHT1=64 #图片的高度
IN_CH1=1 # 输入层通道
OUT_CH1=16  # 第一层卷积输出通道
OUT_CH2=32  # 第二层卷积输出通道
KERNEL_WIDTH3=16 # 第二层池化输出图像宽度
KERNEL_HEIGHT3=16 # 第二层池化输出图像高度
OUT_CH3=128 # 第一层全连接输出通道
OUT_CH4=29 #最终输出通道

KERNEL_WIDTH1=5 #卷积核宽度
KERNEL_HEIGHT1=5 #卷积核高度
KERNEL_WIDTH2=5
KERNEL_HEIGHT2=5

#Conv1 卷积1

print('IN_CH1 is',IN_CH1)

X_STRIDE1=1 #移动步长x
Y_STRIDE1=1 #移动步长y
RELU_EN1 = 1 
MODE1=1    #0:VALID, 1:SAME #选择padding
#MODE1=0    #0:VALID, 1:SAME #选择padding
if(MODE1):
    X_PADDING1=int((KERNEL_WIDTH1-1)/2)
    Y_PADDING1=int((KERNEL_HEIGHT1-1)/2)
else:
    X_PADDING1=0
    Y_PADDING1=0
    
OUT_WIDTH1=int((IN_WIDTH1+2*X_PADDING1-KERNEL_WIDTH1)/X_STRIDE1+1) #输出的宽度
OUT_HEIGHT1=int((IN_HEIGHT1+2*Y_PADDING1-KERNEL_HEIGHT1)/Y_STRIDE1+1) #输出的高度

#Pool1 池化1
MODE11=2  #mode: 0:MEAN, 1:MIN, 2:MAX #选择最大池化
IN_WIDTH11=OUT_WIDTH1 #输入宽度为第一层卷积输出的宽度
IN_HEIGHT11=OUT_HEIGHT1 #输入高度为第一层卷积输出的高度
IN_CH11=OUT_CH1 #

KERNEL_WIDTH11=2
KERNEL_HEIGHT11=2

OUT_CH11=IN_CH11
OUT_WIDTH11=int(IN_WIDTH11/KERNEL_WIDTH11)
OUT_HEIGHT11=int(IN_HEIGHT11/KERNEL_HEIGHT11)

#Conv2 卷积2
IN_WIDTH2=OUT_WIDTH11
IN_HEIGHT2=OUT_HEIGHT11
IN_CH2=OUT_CH11
print('IN_CH2 is',IN_CH2)


X_STRIDE2=1
Y_STRIDE2=1

RELU_EN2=1
MODE2=1  #0:VALID, 1:SAME #选择padding
#MODE2=0  #0:VALID, 1:SAME #选择padding
if(MODE2):
    X_PADDING2=int((KERNEL_WIDTH2-1)/2)
    Y_PADDING2=int((KERNEL_HEIGHT2-1)/2)
else:
    X_PADDING2=0
    Y_PADDING2=0

OUT_WIDTH2=int((IN_WIDTH2+2*X_PADDING2-KERNEL_WIDTH2)/X_STRIDE2+1)
OUT_HEIGHT2=int((IN_HEIGHT2+2*Y_PADDING2-KERNEL_HEIGHT2)/Y_STRIDE2+1)

#Pool2 池化2
MODE21=2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_WIDTH21=OUT_WIDTH2
IN_HEIGHT21=OUT_HEIGHT2
IN_CH21=OUT_CH2

KERNEL_WIDTH21=2
KERNEL_HEIGHT21=2

OUT_CH21=IN_CH21
OUT_WIDTH21=int(IN_WIDTH21/KERNEL_WIDTH21)
OUT_HEIGHT21=int(IN_HEIGHT21/KERNEL_HEIGHT21)

#Fc1 全连接1 由于全连接和卷积层的计算原理相同，所以可利用卷积层的ip实现全连接
IN_WIDTH3=OUT_WIDTH21
IN_HEIGHT3=OUT_HEIGHT21
IN_CH3=OUT_CH21
print('IN_CH3 is',IN_CH3)

X_STRIDE3=1
Y_STRIDE3=1
RELU_EN3=1
MODE3=0  #0:VALID, 1:SAME #选择不padding
if(MODE3):
    X_PADDING3=int((KERNEL_WIDTH3-1/2))
    Y_PADDING3=int((KERNEL_HEIGHT3-1)/2)
else:
    X_PADDING3=0
    Y_PADDING3=0

OUT_WIDTH3=int((IN_WIDTH3+2*X_PADDING3-KERNEL_WIDTH3)/X_STRIDE3+1)
OUT_HEIGHT3=int((IN_HEIGHT3+2*Y_PADDING3-KERNEL_HEIGHT3)/Y_STRIDE3+1)

#Fc2 全连接2
IN_WIDTH4=OUT_WIDTH3
IN_HEIGHT4=OUT_HEIGHT3
IN_CH4=OUT_CH3
print('IN_CH4 is',IN_CH4)

KERNEL_WIDTH4=1
KERNEL_HEIGHT4=1
X_STRIDE4=1
Y_STRIDE4=1

RELU_EN4=1
MODE4=0  #0:VALID, 1:SAME
if(MODE4):
    X_PADDING4=int((KERNEL_WIDTH4-1/2))
    Y_PADDING4=int((KERNEL_HEIGHT4-1)/2)
else:
    X_PADDING4=0
    Y_PADDING4=0

OUT_WIDTH4=int((IN_WIDTH4+2*X_PADDING4-KERNEL_WIDTH4)/X_STRIDE4+1)
OUT_HEIGHT4=int((IN_HEIGHT4+2*Y_PADDING4-KERNEL_HEIGHT4)/Y_STRIDE4+1)


IN_CH1 is 1
IN_CH2 is 16
IN_CH3 is 32
IN_CH4 is 128


In [23]:
#=============================分配存储空间给各个参数===============================#
#input image
image=xlnk.cma_array(shape=(IN_HEIGHT1,IN_WIDTH1,IN_CH1),cacheable=0,dtype=np.float32) #分配空间给image存储图片
#conv1
W_conv1=xlnk.cma_array(shape=(KERNEL_HEIGHT1,KERNEL_WIDTH1,IN_CH1,OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积权重的存储空间
b_conv1=xlnk.cma_array(shape=(OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积偏置的存储空间
h_conv1=xlnk.cma_array(shape=(OUT_HEIGHT1,OUT_WIDTH1,OUT_CH1),cacheable=0,dtype=np.float32) #给池化分配空间
h_pool1=xlnk.cma_array(shape=(OUT_HEIGHT11,OUT_WIDTH11,OUT_CH11),cacheable=0,dtype=np.float32) #给池化分配空间
#conv2
W_conv2=xlnk.cma_array(shape=(KERNEL_HEIGHT2,KERNEL_WIDTH2,IN_CH2,OUT_CH2),cacheable=0,dtype=np.float32)
b_conv2=xlnk.cma_array(shape=(OUT_CH2),cacheable=0,dtype=np.float32)
h_conv2=xlnk.cma_array(shape=(OUT_HEIGHT2,OUT_WIDTH2,OUT_CH2),cacheable=0,dtype=np.float32)
h_pool2=xlnk.cma_array(shape=(OUT_HEIGHT21,OUT_WIDTH21,OUT_CH21),cacheable=0,dtype=np.float32)
#fc1
W_fc1=xlnk.cma_array(shape=(KERNEL_HEIGHT3, KERNEL_WIDTH3, IN_CH3, OUT_CH3),cacheable=0,dtype=np.float32)
b_fc1=xlnk.cma_array(shape=(OUT_CH3),cacheable=0,dtype=np.float32)
h_fc1=xlnk.cma_array(shape=(OUT_HEIGHT3,OUT_WIDTH3,OUT_CH3),cacheable=0,dtype=np.float32)
#fc2
W_fc2=xlnk.cma_array(shape=(KERNEL_HEIGHT4, KERNEL_WIDTH4, IN_CH4, OUT_CH4),cacheable=0,dtype=np.float32)
b_fc2=xlnk.cma_array(shape=(OUT_CH4),cacheable=0,dtype=np.float32)
h_fc2=xlnk.cma_array(shape=(OUT_HEIGHT4,OUT_WIDTH4,OUT_CH4),cacheable=0,dtype=np.float32)


In [24]:
#=============================初始化：读取训练好的参数，并写入分配好的板上存储空间===============================#
w_conv1=np.load('./record/whale_float_5x5/tensor1.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv1=np.load('./record/whale_float_5x5/tensor2.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
# w_conv1=np.transpose(w_conv1,(1,2,3,0))
for i in range(KERNEL_HEIGHT1): #将数据写入板子
    for j in range(KERNEL_WIDTH1):
        for k in range(IN_CH1):
        	for l in range(OUT_CH1):
        		W_conv1[i][j][k][l]=w_conv1[i][j][k][l]
for i in range(OUT_CH1): #将数据写入板子
    b_conv1[i]=B_conv1[i]
#     b_conv1[i]=0
#=================================================    
w_conv2=np.load('./record/whale_float_5x5/tensor3.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv2=np.load('./record/whale_float_5x5/tensor4.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
# w_conv2=np.transpose(w_conv2,(1,2,3,0))
for i in range(KERNEL_HEIGHT2):
    for j in range(KERNEL_WIDTH2):
        for k in range(IN_CH2):
        	for l in range(OUT_CH2):
        		W_conv2[i][j][k][l]=w_conv2[i][j][k][l]
for i in range(OUT_CH2):
    b_conv2[i]=B_conv2[i]
#     b_conv2[i]=0
#=================================================    
w_fc1=np.load('./record/whale_float_5x5/tensor5.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc1=np.load('./record/whale_float_5x5/tensor6.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
# w_fc1=np.transpose(w_fc1,(1,0))
w_fc1=w_fc1.reshape((KERNEL_HEIGHT3,KERNEL_WIDTH3,IN_CH3,OUT_CH3))
for i in range(KERNEL_HEIGHT3):
    for j in range(KERNEL_WIDTH3):
        for k in range(IN_CH3):
        	for l in range(OUT_CH3):
        		W_fc1[i][j][k][l]=w_fc1[i][j][k][l]
for i in range(OUT_CH3):
    b_fc1[i]=B_fc1[i]
#     b_fc1[i]=0
#=================================================    
w_fc2=np.load('./record/whale_float_5x5/tensor7.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc2=np.load('./record/whale_float_5x5/tensor8.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
# w_fc2=np.transpose(w_fc2,(1,0))
w_fc2=w_fc2.reshape((KERNEL_HEIGHT4,KERNEL_WIDTH4,IN_CH4,OUT_CH4))
for i in range(KERNEL_HEIGHT4):
    for j in range(KERNEL_WIDTH4):
        for k in range(IN_CH4):
        	for l in range(OUT_CH4):
        		W_fc2[i][j][k][l]=w_fc2[i][j][k][l]                                      
for i in range(OUT_CH4):
    b_fc2[i]=B_fc2[i]
#     b_fc2[i]=0
#=================================================    
print("Finish initial")

Finish initial


In [39]:
# 单张图片测试
img = cv2.imread("./test_data/whale.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
#start_time,end_time,time_use=0,0,0
for i in range(IN_HEIGHT1):
    for j in range(IN_WIDTH1):
        image[i][j][0]=img[i][j]

#调用卷积ip、池化ip，实现硬件加速
#由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
start_time=time.time()
#conv1
RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
conv1_time=time.time()
RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
pool1_time=time.time()
# conv2
RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
conv2_time=time.time()
RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
pool2_time=time.time()
# fc1 
RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
fc1_time=time.time()
# fc2
RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
fc2_time=time.time()

result=h_fc2[0][0].argmax() 

result_name=name[result]

# 显示图像        
print('result:',result_name)
print('conv1_time:',np.round(conv1_time-start_time,4),'s')
print('pool1_time:',np.round(pool1_time-conv1_time,4),'s')
print('conv2_time:',np.round(conv2_time-pool1_time,4),'s')
print('pool2_time:',np.round(pool2_time-conv2_time,4),'s')
print('fc1  _time:',np.round(fc1_time-pool2_time,4),'s')
print('fc2  _time:',np.round(fc2_time-fc1_time,4),'s')
print('all  _time:',np.round(fc2_time-start_time,4),'s')

result: 瓜头鲸
conv1_time: 0.3222 s
pool1_time: 0.0044 s
conv2_time: 2.4309 s
pool2_time: 0.0028 s
fc1  _time: 0.2052 s
fc2  _time: 0.0021 s
all  _time: 2.9676 s


In [20]:
# for i in range(OUT_CH4):
# #     b_fc2[i]=B_fc2[i]
#     b_fc2[i]=0
# for i in range(OUT_CH3):
# #     b_fc1[i]=B_fc1[i]
#     b_fc1[i]=0
# for i in range(OUT_CH2):
# #     b_conv2[i]=B_conv2[i]
#     b_conv2[i]=0
# for i in range(OUT_CH1): #将数据写入板子
# #     b_conv1[i]=B_conv1[i]
#     b_conv1[i]=0

In [44]:
whale_test_image_label=list()
whale_train_image_label=list()
f1=open('./dataset/whale_test_image_label_6_13.txt','r')
f2=open('./dataset/whale_train_image_label_6_13.txt','r')
for i in f1.readlines():
    whale_test_image_label.append(i.split('\n')[0].split(' '))
for i in f2.readlines():
    whale_train_image_label.append(i.split('\n')[0].split(' '))
f1.close()
f2.close()

In [117]:
ADD_Accurate=0
# process_bar = ShowProcess(100, 'Processing Finished')
for i in range(100):
    img = cv2.resize(cv2.imread("./dataset/whale64x64/"+whale_test_image_label[i][0], cv2.IMREAD_GRAYSCALE),(32,32)).astype(np.float32) #读入一张图片
    res=cnn_mnist_test(img)
    if(res==int(ship_test_image_label[i][1])):
        print(whale_test_image_label[i][0],whale_test_image_label[i][1])
        ADD_Accurate+=1
#     process_bar.show_process()
print('正确率:',ADD_Accurate,'/',100,'=',np.round(ADD_Accurate/100*100,2),'%')

5614d55b93e35b.jpg 0
60443783c1ddcd.jpg 0


KeyboardInterrupt: 

In [60]:
ADD_Accurate

15

In [ ]:
ADD_Accurate=0
process_bar = ShowProcess(len(whale_test_image_label), 'Processing Finished')
for i in range(len(ship_test_image_label)):
    img = cv2.imread("./dataset/whale64x64/"+whale_test_image_label[i][0], cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
    res=cnn_mnist_test(img)
    if(res==int(whale_test_image_label[i][1])):
        ADD_Accurate+=1
    process_bar.show_process()
print('正确率:',ADD_Accurate,'/',len(whale_test_image_label),'=',np.round(ADD_Accurate/len(whale_test_image_label)*100,2),'%')

In [ ]:
ADD_Accurate=0
process_bar = ShowProcess(len(whale_test_image_label), 'Processing Finished')
for i in range(len(ship_test_image_label)):
    img = cv2.imread("./dataset/whale64x64/"+whale_test_image_label[i][0], cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
    res=cnn_mnist_test(img)
    if(res==int(whale_test_image_label[i][1])):
        ADD_Accurate+=1
    process_bar.show_process()
print('正确率:',ADD_Accurate,'/',len(whale_test_image_label),'=',np.round(ADD_Accurate/len(whale_test_image_label)*100,2),'%')

In [ ]:
name=['瓜头鲸']

In [21]:
# 单张图片测试
img = cv2.imread("./test_data/whale.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
#start_time,end_time,time_use=0,0,0
for i in range(IN_HEIGHT1):
    for j in range(IN_WIDTH1):
        image[i][j][0]=img[i][j]

#调用卷积ip、池化ip，实现硬件加速
#由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
start_time=time.time()
#conv1
RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
conv1_time=time.time()
RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
pool1_time=time.time()
# conv2
RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
conv2_time=time.time()
RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
pool2_time=time.time()
# fc1 
RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
fc1_time=time.time()
# fc2
RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
fc2_time=time.time()

result=h_fc2[0][0].argmax() 

result_name=name[result]

# 显示图像        
print('result:',result_name)
print('conv1_time:',np.round(conv1_time-start_time,4),'s')
print('pool1_time:',np.round(pool1_time-conv1_time,4),'s')
print('conv2_time:',np.round(conv2_time-pool1_time,4),'s')
print('pool2_time:',np.round(pool2_time-conv2_time,4),'s')
print('fc1  _time:',np.round(fc1_time-pool2_time,4),'s')
print('fc2  _time:',np.round(fc2_time-fc1_time,4),'s')
print('all  _time:',np.round(fc2_time-start_time,4),'s')

# 3 num

In [5]:
# num
IN_WIDTH1=28 #图片的宽度
IN_HEIGHT1=28 #图片的高度
IN_CH1=1 # 输入层通道
OUT_CH1=6  # 第一层卷积输出通道
OUT_CH2=16  # 第二层卷积输出通道
KERNEL_WIDTH3=7 # 第二层池化输出图像宽度
KERNEL_HEIGHT3=7 # 第二层池化输出图像高度
OUT_CH3=120 # 第一层全连接输出通道
OUT_CH4=10 #最终输出通道

#Conv1 卷积1

print('IN_CH1 is',IN_CH1)
KERNEL_WIDTH1=3 #卷积核宽度
KERNEL_HEIGHT1=3 #卷积核高度
X_STRIDE1=1 #移动步长x
Y_STRIDE1=1 #移动步长y
RELU_EN1 = 1 
MODE1=1    #0:VALID, 1:SAME #选择padding
#MODE1=0    #0:VALID, 1:SAME #选择padding
if(MODE1):
    X_PADDING1=int((KERNEL_WIDTH1-1)/2)
    Y_PADDING1=int((KERNEL_HEIGHT1-1)/2)
else:
    X_PADDING1=0
    Y_PADDING1=0
    
OUT_WIDTH1=int((IN_WIDTH1+2*X_PADDING1-KERNEL_WIDTH1)/X_STRIDE1+1) #输出的宽度
OUT_HEIGHT1=int((IN_HEIGHT1+2*Y_PADDING1-KERNEL_HEIGHT1)/Y_STRIDE1+1) #输出的高度

#Pool1 池化1
MODE11=2  #mode: 0:MEAN, 1:MIN, 2:MAX #选择最大池化
IN_WIDTH11=OUT_WIDTH1 #输入宽度为第一层卷积输出的宽度
IN_HEIGHT11=OUT_HEIGHT1 #输入高度为第一层卷积输出的高度
IN_CH11=OUT_CH1 #

KERNEL_WIDTH11=2
KERNEL_HEIGHT11=2

OUT_CH11=IN_CH11
OUT_WIDTH11=int(IN_WIDTH11/KERNEL_WIDTH11)
OUT_HEIGHT11=int(IN_HEIGHT11/KERNEL_HEIGHT11)

#Conv2 卷积2
IN_WIDTH2=OUT_WIDTH11
IN_HEIGHT2=OUT_HEIGHT11
IN_CH2=OUT_CH11
print('IN_CH2 is',IN_CH2)

KERNEL_WIDTH2=3
KERNEL_HEIGHT2=3
X_STRIDE2=1
Y_STRIDE2=1

RELU_EN2=1
MODE2=1  #0:VALID, 1:SAME #选择padding
#MODE2=0  #0:VALID, 1:SAME #选择padding
if(MODE2):
    X_PADDING2=int((KERNEL_WIDTH2-1)/2)
    Y_PADDING2=int((KERNEL_HEIGHT2-1)/2)
else:
    X_PADDING2=0
    Y_PADDING2=0

OUT_WIDTH2=int((IN_WIDTH2+2*X_PADDING2-KERNEL_WIDTH2)/X_STRIDE2+1)
OUT_HEIGHT2=int((IN_HEIGHT2+2*Y_PADDING2-KERNEL_HEIGHT2)/Y_STRIDE2+1)

#Pool2 池化2
MODE21=2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_WIDTH21=OUT_WIDTH2
IN_HEIGHT21=OUT_HEIGHT2
IN_CH21=OUT_CH2

KERNEL_WIDTH21=2
KERNEL_HEIGHT21=2

OUT_CH21=IN_CH21
OUT_WIDTH21=int(IN_WIDTH21/KERNEL_WIDTH21)
OUT_HEIGHT21=int(IN_HEIGHT21/KERNEL_HEIGHT21)

#Fc1 全连接1 由于全连接和卷积层的计算原理相同，所以可利用卷积层的ip实现全连接
IN_WIDTH3=OUT_WIDTH21
IN_HEIGHT3=OUT_HEIGHT21
IN_CH3=OUT_CH21
print('IN_CH3 is',IN_CH3)

X_STRIDE3=1
Y_STRIDE3=1
RELU_EN3=1
MODE3=0  #0:VALID, 1:SAME #选择不padding
if(MODE3):
    X_PADDING3=int((KERNEL_WIDTH3-1/2))
    Y_PADDING3=int((KERNEL_HEIGHT3-1)/2)
else:
    X_PADDING3=0
    Y_PADDING3=0

OUT_WIDTH3=int((IN_WIDTH3+2*X_PADDING3-KERNEL_WIDTH3)/X_STRIDE3+1)
OUT_HEIGHT3=int((IN_HEIGHT3+2*Y_PADDING3-KERNEL_HEIGHT3)/Y_STRIDE3+1)

#Fc2 全连接2
IN_WIDTH4=OUT_WIDTH3
IN_HEIGHT4=OUT_HEIGHT3
IN_CH4=OUT_CH3
print('IN_CH4 is',IN_CH4)

KERNEL_WIDTH4=1
KERNEL_HEIGHT4=1
X_STRIDE4=1
Y_STRIDE4=1

RELU_EN4=1
MODE4=0  #0:VALID, 1:SAME
if(MODE4):
    X_PADDING4=int((KERNEL_WIDTH4-1/2))
    Y_PADDING4=int((KERNEL_HEIGHT4-1)/2)
else:
    X_PADDING4=0
    Y_PADDING4=0

OUT_WIDTH4=int((IN_WIDTH4+2*X_PADDING4-KERNEL_WIDTH4)/X_STRIDE4+1)
OUT_HEIGHT4=int((IN_HEIGHT4+2*Y_PADDING4-KERNEL_HEIGHT4)/Y_STRIDE4+1)


IN_CH1 is 1
IN_CH2 is 6
IN_CH3 is 16
IN_CH4 is 120


In [6]:
#=============================分配存储空间给各个参数===============================#
#input image
image=xlnk.cma_array(shape=(IN_HEIGHT1,IN_WIDTH1,IN_CH1),cacheable=0,dtype=np.float32) #分配空间给image存储图片
#conv1
W_conv1=xlnk.cma_array(shape=(KERNEL_HEIGHT1,KERNEL_WIDTH1,IN_CH1,OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积权重的存储空间
b_conv1=xlnk.cma_array(shape=(OUT_CH1),cacheable=0,dtype=np.float32) #分配第一层卷积偏置的存储空间
h_conv1=xlnk.cma_array(shape=(OUT_HEIGHT1,OUT_WIDTH1,OUT_CH1),cacheable=0,dtype=np.float32) #给池化分配空间
h_pool1=xlnk.cma_array(shape=(OUT_HEIGHT11,OUT_WIDTH11,OUT_CH11),cacheable=0,dtype=np.float32) #给池化分配空间
#conv2
W_conv2=xlnk.cma_array(shape=(KERNEL_HEIGHT2,KERNEL_WIDTH2,IN_CH2,OUT_CH2),cacheable=0,dtype=np.float32)
b_conv2=xlnk.cma_array(shape=(OUT_CH2),cacheable=0,dtype=np.float32)
h_conv2=xlnk.cma_array(shape=(OUT_HEIGHT2,OUT_WIDTH2,OUT_CH2),cacheable=0,dtype=np.float32)
h_pool2=xlnk.cma_array(shape=(OUT_HEIGHT21,OUT_WIDTH21,OUT_CH21),cacheable=0,dtype=np.float32)
#fc1
W_fc1=xlnk.cma_array(shape=(KERNEL_HEIGHT3, KERNEL_WIDTH3, IN_CH3, OUT_CH3),cacheable=0,dtype=np.float32)
b_fc1=xlnk.cma_array(shape=(OUT_CH3),cacheable=0,dtype=np.float32)
h_fc1=xlnk.cma_array(shape=(OUT_HEIGHT3,OUT_WIDTH3,OUT_CH3),cacheable=0,dtype=np.float32)
#fc2
W_fc2=xlnk.cma_array(shape=(KERNEL_HEIGHT4, KERNEL_WIDTH4, IN_CH4, OUT_CH4),cacheable=0,dtype=np.float32)
b_fc2=xlnk.cma_array(shape=(OUT_CH4),cacheable=0,dtype=np.float32)
h_fc2=xlnk.cma_array(shape=(OUT_HEIGHT4,OUT_WIDTH4,OUT_CH4),cacheable=0,dtype=np.float32)


In [7]:
#=============================初始化：读取训练好的参数，并写入分配好的板上存储空间===============================#
w_conv1=np.load('./record/num/tensor1.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv1=np.load('./record/num/tensor2.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv1=np.transpose(w_conv1,(1,2,3,0))
for i in range(KERNEL_HEIGHT1): #将数据写入板子
    for j in range(KERNEL_WIDTH1):
        for k in range(IN_CH1):
        	for l in range(OUT_CH1):
        		W_conv1[i][j][k][l]=w_conv1[i][j][k][l]
for i in range(OUT_CH1): #将数据写入板子
#     b_conv1[i]=B_conv1[i]
    b_conv1[i]=0
#=================================================    
w_conv2=np.load('./record/num/tensor3.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_conv2=np.load('./record/num/tensor4.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_conv2=np.transpose(w_conv2,(1,2,3,0))
for i in range(KERNEL_HEIGHT2):
    for j in range(KERNEL_WIDTH2):
        for k in range(IN_CH2):
        	for l in range(OUT_CH2):
        		W_conv2[i][j][k][l]=w_conv2[i][j][k][l]
for i in range(OUT_CH2):
#     b_conv2[i]=B_conv2[i]
    b_conv2[i]=0
#=================================================    
w_fc1=np.load('./record/num/tensor5.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc1=np.load('./record/num/tensor6.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc1=np.transpose(w_fc1,(1,0))
w_fc1=w_fc1.reshape((KERNEL_HEIGHT3,KERNEL_WIDTH3,IN_CH3,OUT_CH3))
for i in range(KERNEL_HEIGHT3):
    for j in range(KERNEL_WIDTH3):
        for k in range(IN_CH3):
        	for l in range(OUT_CH3):
        		W_fc1[i][j][k][l]=w_fc1[i][j][k][l]
for i in range(OUT_CH3):
#     b_fc1[i]=B_fc1[i]
    b_fc1[i]=0
#=================================================    
w_fc2=np.load('./record/num/tensor7.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
B_fc2=np.load('./record/num/tensor8.npy', encoding='bytes',allow_pickle=True) #读取第一层卷积的权重
w_fc2=np.transpose(w_fc2,(1,0))
w_fc2=w_fc2.reshape((KERNEL_HEIGHT4,KERNEL_WIDTH4,IN_CH4,OUT_CH4))
for i in range(KERNEL_HEIGHT4):
    for j in range(KERNEL_WIDTH4):
        for k in range(IN_CH4):
        	for l in range(OUT_CH4):
        		W_fc2[i][j][k][l]=w_fc2[i][j][k][l]                                      
for i in range(OUT_CH4):
#     b_fc2[i]=B_fc2[i]
    b_fc2[i]=0
#=================================================    
print("Finish initial")

Finish initial


In [11]:
# 单张图片测试
img = cv2.imread("./test_data/num.png", cv2.IMREAD_GRAYSCALE).astype(np.float32) #读入一张图片
#start_time,end_time,time_use=0,0,0
for i in range(IN_HEIGHT1):
    for j in range(IN_WIDTH1):
        image[i][j][0]=img[i][j]

#调用卷积ip、池化ip，实现硬件加速
#由于参数是可以重新定义的，所以可对卷积、池化的ip进行复用
start_time=time.time()
#conv1
RunConv(conv,KERNEL_WIDTH1,KERNEL_HEIGHT1,X_STRIDE1,Y_STRIDE1,MODE1,RELU_EN1,image,W_conv1,b_conv1,h_conv1)
conv1_time=time.time()
RunPool(pool, KERNEL_WIDTH11, KERNEL_HEIGHT11, MODE11, h_conv1, h_pool1)#池化层是为了缩小图片大小，故没有权重和偏置参数
pool1_time=time.time()
# conv2
RunConv(conv, KERNEL_WIDTH2, KERNEL_HEIGHT2, X_STRIDE2, Y_STRIDE2, MODE2, RELU_EN2, h_pool1, W_conv2, b_conv2,h_conv2)
conv2_time=time.time()
RunPool(pool, KERNEL_WIDTH21, KERNEL_HEIGHT21, MODE21, h_conv2, h_pool2)
pool2_time=time.time()
# fc1 
RunConv(conv, KERNEL_WIDTH3, KERNEL_HEIGHT3, X_STRIDE3, Y_STRIDE3, MODE3, RELU_EN3, h_pool2, W_fc1, b_fc1,h_fc1)
fc1_time=time.time()
# fc2
RunConv(conv, KERNEL_WIDTH4, KERNEL_HEIGHT4, X_STRIDE4, Y_STRIDE4, MODE4, RELU_EN4, h_fc1, W_fc2, b_fc2,h_fc2)#h_fc2即为推理的最终结果
fc2_time=time.time()

result=h_fc2[0][0].argmax() 
# 显示结果
print('result:',result)
print('conv1_time:',np.round(conv1_time-start_time,4),'s')
print('pool1_time:',np.round(pool1_time-conv1_time,4),'s')
print('conv2_time:',np.round(conv2_time-pool1_time,4),'s')
print('pool2_time:',np.round(pool2_time-conv2_time,4),'s')
print('fc1  _time:',np.round(fc1_time-pool2_time,4),'s')
print('fc2  _time:',np.round(fc2_time-fc1_time,4),'s')
print('all  _time:',np.round(fc2_time-start_time,4),'s')

result: 5
conv1_time: 0.0099 s
pool1_time: 0.0013 s
conv2_time: 0.032 s
pool2_time: 0.0012 s
fc1  _time: 0.0196 s
fc2  _time: 0.0016 s
all  _time: 0.0656 s


In [10]:
time_add=0
num_accuracy = [0 for x in range(0,10)]
add=0

# 读取图片-数字识别
process_bar = ShowProcess(1000, 'Processing Finished')
for i in range(10):
    for j in range(100):
        img = cv2.imread("./test_data/{}/{}.png".format(i,j), cv2.IMREAD_GRAYSCALE).astype(np.int64) #读入一张图片
        res=cnn_mnist_test(img) 
        time_add+=res[1]
        if res[0]==i:
            num_accuracy[i]+=1
        process_bar.show_process()
    

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [11]:
# 结果显示
add=0
print('MODEL:3x3_6_16_120')
print('Quantify:YES-1')
print('input/output:int32 weights:int8 bits:int8 inner:int64')
print('Frequency:250MHz')
print('===========')
print('processing pictures are 1000')
print('===========')
for i in range(10):
    print('num {} accuracy is:{}%'.format(i,num_accuracy[i]))
    add += num_accuracy[i]
print('===========')
print('all accuracy is:',round(add/10,1),'%')
print('===========')
print('all num processing use time is',np.round(time_add,2),'s')
print('single num processing use time is',np.round(time_add/1000,4),'s')

MODEL:3x3_6_16_120
Quantify:YES-1
input/output:int32 weights:int8 bits:int8 inner:int64
Frequency:250MHz
processing pictures are 1000
num 0 accuracy is:100%
num 1 accuracy is:99%
num 2 accuracy is:96%
num 3 accuracy is:100%
num 4 accuracy is:100%
num 5 accuracy is:100%
num 6 accuracy is:100%
num 7 accuracy is:100%
num 8 accuracy is:99%
num 9 accuracy is:96%
all accuracy is: 99.0 %
all num processing use time is 44.02 s
single num processing use time is 0.044 s


In [ ]:
每一层的时间相加=有效运行时间